# HW2.0:
How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
What is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?

## Answer HW2.0
By doing order inversion or value-to-key conversions.  This is used in memory in MapReduce.  Combiner performs further aggregations after the mapper function, thereby reducing the memory needed by the reducer if it is to perform any in-memory processing. "It worth noting that Combiners can be used in this schema to exclude duplicates from category lists before data will be transmitted to Reducer." The Hadoop shuffle is when the partitioners group the keys , value pairs by keys before sending them to the reducer.

# HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

## Answer HW#2.1

In [1]:
%%writefile km_hw2_1_counters.py
# MRJob development
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv 

cols = 'ComplaintID,Product,Sub-product,Issue,Subissue,State,ZIPcode,Submittedvia,Datereceived,Datesent,Company,Companyresponse,Timelyresponse,Consumerdisputed'.split(',')
print cols

class MRcounters(MRJob):
    
    def mapper(self, _, line):
        row = dict(zip(cols, [ a.strip() for a in csv.reader([line]).next()]))
        #self.increment_counter('group', 'Num_mapper_calls', 1)

        if row['Product'] == 'Product':
            return
        
        elif row['Product'].lower() == 'debt collection':
            self.increment_counter('debt collection' , 'Num_mapper_calls' , 1)
                
        elif row['Product'].lower() == 'mortgage':
            self.increment_counter('mortgage' , 'Num_mapper_calls' , 1)
                
        else :
            self.increment_counter('other' , 'Num_mapper_calls' , 1) 

if __name__ == "__main__":
    MRcounters.run()


Overwriting km_hw2_1_counters.py


In [2]:
# MRJob rund code
!python km_hw2_1_counters.py Consumer_Complaints.csv

['ComplaintID', 'Product', 'Sub-product', 'Issue', 'Subissue', 'State', 'ZIPcode', 'Submittedvia', 'Datereceived', 'Datesent', 'Company', 'Companyresponse', 'Timelyresponse', 'Consumerdisputed']


No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.z001c9v.20160729.140536.621000
Running step 1 of 1...
Counters: 3
	debt collection
		Num_mapper_calls=44372
	mortgage
		Num_mapper_calls=125752
	other
		Num_mapper_calls=142788
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.z001c9v.20160729.140536.621000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.z001c9v.20160729.140536.621000...


No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.Z001C9V.20160706.040015.428000
Running step 1 of 1...

>Counters: 3
	1. debt collection
		Num_mapper_calls=44372
	2. mortgage
		Num_mapper_calls=125752
	3. other
		Num_mapper_calls=142788
       
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.Z001C9V.20160706.040015.428000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw2_1_counters.Z001C9V.20160706.040015.428000...

# HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

### Answer HW#2.2 - a
Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

In [3]:
!echo foo,foo,quux,labs,foo,bar,quux > input_hw_2_2.txt

In [4]:
%%writefile km_hw_2_2_counters_a.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import string


class MRCounter2_2(MRJob):
    counter = 0 
    def mapper(self, _ , line):
        self.increment_counter('group', 'Mapper_Call_Num' , 1) 
        word = line.strip().lower().split(',')
        #word = line.split('\t')
        print (word)
        for item in word:
            yield (item,1)
           
    def reducer(self, key, value):
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        yield(key, sum(value))

        #for w in word

if __name__ == '__main__':
    MRCounter2_2.run()

Overwriting km_hw_2_2_counters_a.py


In [5]:
!python km_hw_2_2_counters_a.py input_hw_2_2.txt

['foo', 'foo', 'quux', 'labs', 'foo', 'bar', 'quux']
"bar"	1
"foo"	3
"labs"	1
"quux"	2


No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_a.z001c9v.20160729.140544.778000
Running step 1 of 1...
Counters: 1
	group
		Mapper_Call_Num=1
Counters: 2
	group
		Mapper_Call_Num=1
		Reducer_Call_Num=4
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_a.z001c9v.20160729.140544.778000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_a.z001c9v.20160729.140544.778000...


In [6]:
# One run through the mapper creates a list that holds 4 keys and 1s.  The reducer receives a list of values one key at a time. 
# Thus, since there are 4 unique keys(words) in the list, the reducer runs 4 times.

### Answer HW#2.2 - b
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.

In [7]:
%%writefile km_hw_2_2_counters_b.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv , string, re


class MRCounter2_2(MRJob):
    counter = 0 
    def mapper(self, _ , line):
        self.increment_counter('group', 'Mapper_Call_Num' , 1) 
        word = line.strip().lower().split(',')
        #word = line.split('\t')
        #print (word)
        issue = word[3]
        #print('issue:', issue)
        word2 = issue.split(' ')
        #print ('word2:' , word2)
        for item in word2:
            #print item
            yield (item,1)
            #break   
    def reducer(self, key, value):
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        yield(key, sum(value))

if __name__ == '__main__':
    MRCounter2_2.run()

Overwriting km_hw_2_2_counters_b.py


In [8]:
!python km_hw_2_2_counters_b.py Consumer_Complaints.csv

""	12549
"/"	12386
"\"account"	16205
"\"application"	8625
"\"loan"	107254
"\"making/receiving"	3226
"a"	3503
"account"	4476
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	243
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can't"	1999
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing/cancelling"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company's"	4858
"cont'd"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	50894
"credited"	92
"customer"	2734
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"didn't"	925
"disclosure"	5214
"disclosures"	64
"dispute"	904
"disputes"	6938
"embezzlement"	3276
"expect"	807
"f

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.z001c9v.20160729.140547.766000
Running step 1 of 1...
Counters: 1
	group
		Mapper_Call_Num=312913
Counters: 2
	group
		Mapper_Call_Num=312913
		Reducer_Call_Num=172
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.z001c9v.20160729.140547.766000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.z001c9v.20160729.140547.766000...


### Output:
No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.Z001C9V.20160706.162736.861000
Running step 1 of 1...
>Counters: 1
	1. group
		Mapper_Call_Num=312913
>Counters: 2
	1. group
		Mapper_Call_Num=312913
		Reducer_Call_Num=172
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.Z001C9V.20160706.162736.861000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_b.Z001C9V.20160706.162736.861000...


### Answer HW#2.2 - c
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job. 


In [9]:
%%writefile km_hw_2_2_counters_c.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv , string, re


class MRCounter2_2(MRJob):
    counter = 0 
    def mapper(self, _ , line):
        self.increment_counter('group', 'Mapper_Call_Num' , 1) 
        word = line.strip().lower().split(',')
        #word = line.split('\t')
        #print (word)
        issue = word[3]
        #print('issue:', issue)
        word2 = issue.split(' ')
        #print ('word2:' , word2)
        for item in word2:
            #print item
            yield (item,1)
            #break 
    def combiner(self, key, value):
        self.increment_counter('group' , 'Combiner_Call_Num' , 1)
        yield (key , sum(value))
        
    def reducer(self, key, value):
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        yield(key, sum(value))

if __name__ == '__main__':
    MRCounter2_2.run()

Overwriting km_hw_2_2_counters_c.py


In [10]:
!python km_hw_2_2_counters_c.py Consumer_Complaints.csv

""	12549
"/"	12386
"\"account"	16205
"\"application"	8625
"\"loan"	107254
"\"making/receiving"	3226
"a"	3503
"account"	4476
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	243
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can't"	1999
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing/cancelling"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company's"	4858
"cont'd"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	50894
"credited"	92
"customer"	2734
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"didn't"	925
"disclosure"	5214
"disclosures"	64
"dispute"	904
"disputes"	6938
"embezzlement"	3276
"expect"	807
"f

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.z001c9v.20160729.140602.047000
Running step 1 of 1...
Counters: 2
	group
		Combiner_Call_Num=319
		Mapper_Call_Num=312913
Counters: 3
	group
		Combiner_Call_Num=319
		Mapper_Call_Num=312913
		Reducer_Call_Num=172
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.z001c9v.20160729.140602.047000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.z001c9v.20160729.140602.047000...


### Output:
No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.Z001C9V.20160706.164411.857000
Running step 1 of 1...
>Counters: 2
	1. group
		Combiner_Call_Num=319
		Mapper_Call_Num=312913
>Counters: 3
	1. group
		Combiner_Call_Num=319
		Mapper_Call_Num=312913
		Reducer_Call_Num=172
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.Z001C9V.20160706.164411.857000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_counters_c.Z001C9V.20160706.164411.857000...


# Answer HW2.2.1
Using a single reducer perform a sort of the words in decreasing order of word counts. Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.

In [11]:
%%writefile km_hw_2_2_1_counters.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv , string, re
#regex = re.compile('[%s]' % re.escape(string.punctuation))

class MRCounter2_2(MRJob):
    def steps(self):
        return [
            MRStep ( mapper = self.mapper , 
                     combiner = self.combiner , 
                     reducer = self.reducer ) ,
            MRStep ( mapper = self.mapper2 ,
                    reducer = self.reducer2 )
        ]

    def __init__(self , *args, **kargs):
        super(MRCounter2_2,self).__init__(*args, **kargs)
        
    def mapper(self, _ , line):
        self.increment_counter('group', 'Mapper_Call_Num' , 1)        
        word = line.strip().lower().split(',')
        #word = line.split('\t')
        #print (word)
        issue = word[3]
        issue = re.sub('\W+',' ',issue) 
        #print('issue:', issue)
        word2 = issue.split(' ')
        #print ('word2:' , word2)
        for item in word2:
            #print item
            if item == u' ':
                yield 'blankxxxx' , 1
            yield (item,1)
            #break 
    def combiner(self, key, value):
        self.increment_counter('group' , 'Combiner_Call_Num' , 1)
        yield (key , sum(value))
        
    def reducer(self, key, value):
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        #output = {}
        yield key , sum(value)
        #output[key] = value
        #yield None , output
        
    def mapper2(self, key , value):
        self.increment_counter('group' , 'Mapper2_Call_Num' ,1)
        num_int = format("%010d" % value)
        #num_sort = num_int + key
        #print 'mapper2:' , (num_int , key)
        #print '\n'
        #yield 'None' , list(num_int) , key
        yield 'None' , (num_int , key)
        
        # yield None , (key , value)
        '''
        for word, num in value:
            #num_int = format("%010d" % num)
            #print(num_int)
            yield num , word        
        '''

        
    '''
        for number in value:
            num_int = str(number)
            print( "'{num_int:0{max_num}d}'.".format(number=number, max_num=max_num) )
            #z10 = "'{number:0{max_num}d}'".format(number=number, max_num=max_num)
            #z10 = str(z10)
            #print number , key 
    '''
    def reducer2(self, key , value):
        self.increment_counter('group' , 'Reducer2_Call_Num' , 1)
        #for word, num in value:
        #    yield num , word
        #print 'reducer2:' , (key , list(value) )
        final_list = []
        for p in value :
            final_list.append(p)
            final_list_bot = final_list[0:9]
            final_list_top = final_list[-51:]
        
        for num , word in final_list_bot : 
            yield 'Bottom List:' , (num , word)
        for num , word in final_list_top :
            yield 'Top List:' , (num , word)
            
        #yield final_list[-51:]
        #yield '\n'
        #yield final_list[0:9]
            
        #print final_list[-51:]
        #print '\n'
        #print final_list[0:9]



if __name__ == '__main__':
    MRCounter2_2.run()

Overwriting km_hw_2_2_1_counters.py


In [12]:
!python km_hw_2_2_1_counters.py Consumer_Complaints.csv

"Bottom List:"	["0000000064", "disclosures"]
"Bottom List:"	["0000000064", "missing"]
"Bottom List:"	["0000000071", "amt"]
"Bottom List:"	["0000000071", "day"]
"Bottom List:"	["0000000075", "checks"]
"Bottom List:"	["0000000075", "convenience"]
"Bottom List:"	["0000000092", "credited"]
"Bottom List:"	["0000000092", "payment"]
"Bottom List:"	["0000000098", "amount"]
"Top List:"	["0000004729", "identity"]
"Top List:"	["0000004858", "company"]
"Top List:"	["0000004858", "investigation"]
"Top List:"	["0000004858", "s"]
"Top List:"	["0000005006", "managing"]
"Top List:"	["0000005214", "disclosure"]
"Top List:"	["0000005214", "verification"]
"Top List:"	["0000005505", "process"]
"Top List:"	["0000005663", "being"]
"Top List:"	["0000005663", "by"]
"Top List:"	["0000005663", "caused"]
"Top List:"	["0000005663", "funds"]
"Top List:"	["0000005663", "low"]
"Top List:"	["0000006248", "the"]
"Top List:"	["0000006337", "lease"]
"Top List:"	["0000006559", "reporting"]
"Top List:"	["0000006920", "comm

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140614.459000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=320
		Mapper_Call_Num=312913
Counters: 3
	group
		Combiner_Call_Num=320
		Mapper_Call_Num=312913
		Reducer_Call_Num=173
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num=173
Counters: 2
	group
		Mapper2_Call_Num=173
		Reducer2_Call_Num=1
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140614.459000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140614.459000...


### Answer HW2.2.2

In [13]:
!python km_hw_2_2_1_counters.py --jobconf mapred.reduce.tasks=3 Consumer_Complaints.csv

"Bottom List:"	["0000000064", "disclosures"]
"Bottom List:"	["0000000064", "missing"]
"Bottom List:"	["0000000071", "amt"]
"Bottom List:"	["0000000071", "day"]
"Bottom List:"	["0000000075", "checks"]
"Bottom List:"	["0000000075", "convenience"]
"Bottom List:"	["0000000092", "credited"]
"Bottom List:"	["0000000092", "payment"]
"Bottom List:"	["0000000098", "amount"]
"Top List:"	["0000004729", "identity"]
"Top List:"	["0000004858", "company"]
"Top List:"	["0000004858", "investigation"]
"Top List:"	["0000004858", "s"]
"Top List:"	["0000005006", "managing"]
"Top List:"	["0000005214", "disclosure"]
"Top List:"	["0000005214", "verification"]
"Top List:"	["0000005505", "process"]
"Top List:"	["0000005663", "being"]
"Top List:"	["0000005663", "by"]
"Top List:"	["0000005663", "caused"]
"Top List:"	["0000005663", "funds"]
"Top List:"	["0000005663", "low"]
"Top List:"	["0000006248", "the"]
"Top List:"	["0000006337", "lease"]
"Top List:"	["0000006559", "reporting"]
"Top List:"	["0000006920", "comm

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140645.011000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=320
		Mapper_Call_Num=312913
Counters: 3
	group
		Combiner_Call_Num=320
		Mapper_Call_Num=312913
		Reducer_Call_Num=173
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num=173
Counters: 2
	group
		Mapper2_Call_Num=173
		Reducer2_Call_Num=1
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140645.011000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_hw_2_2_1_counters.z001c9v.20160729.140645.011000...


I see the same answer set as above.

### Answer HW#2.2.3 - skipped

# HW2.3: Shopping Cart Analysis
   >Do some exploratory data analysis of this dataset guided by the following questions:. 
   >How many unique items are available from this supplier?
   >Using a single reducer: Report your findings: such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,   (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.
data: https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

In [14]:
%%writefile km_2_3_shoppingcart_top_50.py 
from mrjob.job import MRJob
from mrjob.step import MRStep
import string , re 

class ShoppingBag_top50(MRJob) :
    SORT_VALUES = True
    def steps(self):
        return [
            MRStep ( mapper = self.mapper , 
                     combiner = self.combiner , 
                     reducer = self.reducer ) ,
            MRStep ( mapper = self.mapper2 ,
                    reducer = self.reducer2 )
        ]

    def __init__(self , *args, **kargs):
        super(ShoppingBag_top50,self).__init__(*args, **kargs)

    def mapper(self , _ , line) :
        self.increment_counter('group' , 'Mapper_Call_Num' , 1)
        
        bags = line.strip().split(' ')
        #basketsize = len(bags)
        #print bags , len(bags)
        for item in bags:
            #print item , 1
            yield item , 1
            
    def combiner(self, key, value):
        self.increment_counter('group' , 'Combiner_Call_Num' , 1)
        yield (key , sum(value))
        
    def reducer(self, key, value):
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        #output = {}
        yield key , sum(value)
        #output[key] = value
        #yield None , output
        
    def mapper2(self, key , value):
        self.increment_counter('group' , 'Mapper2_Call_Num' ,1)
        num_int = format("%010d" % value)
        #num_sort = num_int + key
        #print 'mapper2:' , (num_int , key)
        #print '\n'
        #yield 'None' , list(num_int) , key
        yield 'None' , (num_int , key)
        
        # yield None , (key , value)


    def reducer2(self, key , value):
        self.increment_counter('group' , 'Reducer2_Call_Num' , 1)      
        #for word, num in value:
        #    yield num , word
        #print 'reducer2:' , (key , list(value) )
        final_list = []
        for p in value :
            final_list.append(p)
            final_list_bot = final_list[0:9]
            final_list_top = final_list[-51:]
        
        for num , word in final_list_bot : 
            yield 'Bottom List:' , (num , word)
        for num , word in final_list_top :
            yield 'Top List:' , (num , word)
            
        #yield final_list[-51:]
        #yield '\n'
        #yield final_list[0:9]
            
        #print final_list[-51:]
        #print '\n'
        #print final_list[0:9]

       
            
if __name__ == '__main__' :
    ShoppingBag_top50.run()
    

Overwriting km_2_3_shoppingcart_top_50.py


In [15]:
!python km_2_3_shoppingcart_top_50.py ProductPurchaseData.txt

"Bottom List:"	["0000000001", "DAI11257"]
"Bottom List:"	["0000000001", "DAI11273"]
"Bottom List:"	["0000000001", "DAI11375"]
"Bottom List:"	["0000000001", "DAI11582"]
"Bottom List:"	["0000000001", "DAI11707"]
"Bottom List:"	["0000000001", "DAI11946"]
"Bottom List:"	["0000000001", "DAI11995"]
"Bottom List:"	["0000000001", "DAI12139"]
"Bottom List:"	["0000000001", "DAI12152"]
"Top List:"	["0000001197", "ELE92920"]
"Top List:"	["0000001214", "GRO85051"]
"Top List:"	["0000001219", "DAI22896"]
"Top List:"	["0000001220", "GRO81087"]
"Top List:"	["0000001261", "DAI31081"]
"Top List:"	["0000001275", "GRO15017"]
"Top List:"	["0000001289", "ELE91337"]
"Top List:"	["0000001290", "DAI43223"]
"Top List:"	["0000001295", "SNA96271"]
"Top List:"	["0000001311", "ELE59935"]
"Top List:"	["0000001316", "DAI88807"]
"Top List:"	["0000001316", "ELE74482"]
"Top List:"	["0000001321", "GRO61133"]
"Top List:"	["0000001345", "ELE56788"]
"Top List:"	["0000001352", "GRO38814"]
"Top List:"	["0000001390", "SNA90094"

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140659.993000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
Counters: 3
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
		Reducer_Call_Num=12592
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num=12592
Counters: 2
	group
		Mapper2_Call_Num=12592
		Reducer2_Call_Num=1
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140659.993000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140659.993000...


In [16]:
%%writefile km_2_3_shoppingcart_uniques.py 
from mrjob.job import MRJob
from mrjob.step import MRStep
import string , re 

class ShoppingBag_uniques(MRJob) :
    def steps(self) :
        return [
                MRStep( mapper = self.mapper ,
                        combiner = self.combiner , 
                        reducer = self.reducer
                      ) 
            ,
                MRStep( mapper = self.mapper2 ,
              #          combiner = self.combiner2 , 
                        reducer = self.reducer2
                      )      
                ]
        
    def __init__(self, *args, **kargs) :
        super(ShoppingBag_uniques, self).__init__(*args,**kargs)
        
    def mapper(self , _ , line) :
        self.increment_counter("group" , "Mapper_Call_Num" , 1)
        
        bags = line.strip().split(' ')
        #basketsize = len(bags)
        #print bags , len(bags)
        for item in bags:
            yield item , 1
            
    def combiner(self , item , value) :
        self.increment_counter("group" , "Combiner_Call_Num" , 1)
        yield item , sum(value)
        
    def reducer(self , item , value) :
        self.increment_counter("group" , "Reducer_Call_Num" , 1)
        yield item , sum(value)
        yield '*' , 1
       
    def mapper2(self , item , value) :
        self.increment_counter("group" , "Mapper2_Call_Num:" , 1) 
        yield item , value
        
    def reducer2(self , item , value ) :
        self.increment_counter("group" , "Reducer2_Call_Num:" , 1)
        total = 0
        for values in value:
            if item == '*' : 
                total += values
            else : return
        yield 'Total Uniques:' , total
            
if __name__ == '__main__' :
    ShoppingBag_uniques.run()

Overwriting km_2_3_shoppingcart_uniques.py


In [17]:
!python km_2_3_shoppingcart_uniques.py ProductPurchaseData.txt

"Total Uniques:"	12592


No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140705.030000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
Counters: 3
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
		Reducer_Call_Num=12592
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num:=25184
Counters: 2
	group
		Mapper2_Call_Num:=25184
		Reducer2_Call_Num:=12593
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140705.030000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140705.030000...


In [18]:
%%writefile km_2_3_shoppingcart_largest.py 
from mrjob.job import MRJob
from mrjob.step import MRStep
import string , re 

class ShoppingBag_largest(MRJob) :
    #def step(self) :
    def __init__(self, *args, **kargs) :
        super(ShoppingBag_largest, self).__init__(*args,**kargs)
        self.maxvalue = ['' , 0]
        
    def mapper(self , _ , line) :
        self.increment_counter('group' , 'Mapper_Call_Num' , 1)
        
        bags = line.strip().split(' ')
        basketsize = len(bags)
        #print bags , len(bags)
        yield bags , basketsize
        #for item in bags:
         #   print item , 1
            
    def reducer(self , key , value) :
        self.increment_counter("group" , "Reducer_Call_Num" , 1)
        test = 0
        #maxvalue = ['',0]
        for v in value:
            test = v 
            #print self.maxvalue[1]
            if test <= self.maxvalue[1] :
                return
            else :
                self.maxvalue[0] = key
                self.maxvalue[1] = test
            
        yield 'Largest Basket:' , self.maxvalue


            
if __name__ == '__main__' :
    ShoppingBag_largest.run()

Overwriting km_2_3_shoppingcart_largest.py


In [19]:
!python km_2_3_shoppingcart_largest.py ProductPurchaseData.txt

"Largest Basket:"	[["DAI11223", "ELE54102", "SNA56249", "SNA30755", "FRO80039", "SNA53220"], 6]
"Largest Basket:"	[["DAI11238", "SNA82274", "SNA96466", "GRO88324", "SNA43409", "FRO35729", "GRO83463", "GRO30912", "ELE34234", "ELE26753", "ELE45560", "ELE99887", "ELE23393", "SNA31446", "SNA40784", "GRO71621"], 16]
"Largest Basket:"	[["DAI11927", "FRO60829", "FRO75586", "ELE49863", "SNA91739", "GRO76157", "FRO73019", "SNA66583", "ELE30182", "DAI22240", "GRO91660", "DAI91705", "GRO81087", "SNA92303", "DAI50189", "GRO48219", "GRO61273"], 17]
"Largest Basket:"	[["DAI12680", "FRO91015", "ELE53126", "SNA56249", "ELE37770", "DAI70456", "SNA63157", "FRO14026", "DAI55911", "SNA37803", "GRO19500", "GRO46854", "FRO40251", "FRO77785", "ELE62598", "DAI75981", "SNA80324", "FRO75586"], 18]
"Largest Basket:"	[["DAI13776", "SNA90094", "GRO44993", "DAI43868", "SNA82528", "FRO73056", "GRO30386", "FRO98184", "SNA96271", "GRO16201", "SNA87654", "SNA61957", "GRO50132", "GRO64900", "FRO62184", "DAI62779", "DAI8

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140710.462000
Running step 1 of 1...
Counters: 1
	group
		Mapper_Call_Num=31101
Counters: 2
	group
		Mapper_Call_Num=31101
		Reducer_Call_Num=30697
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140710.462000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140710.462000...


## 2.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 


In [20]:
!python km_2_3_shoppingcart_top_50.py --jobconf mapred.reduce.tasks=2 ProductPurchaseData.txt

"Bottom List:"	["0000000001", "DAI11257"]
"Bottom List:"	["0000000001", "DAI11273"]
"Bottom List:"	["0000000001", "DAI11375"]
"Bottom List:"	["0000000001", "DAI11582"]
"Bottom List:"	["0000000001", "DAI11707"]
"Bottom List:"	["0000000001", "DAI11946"]
"Bottom List:"	["0000000001", "DAI11995"]
"Bottom List:"	["0000000001", "DAI12139"]
"Bottom List:"	["0000000001", "DAI12152"]
"Top List:"	["0000001197", "ELE92920"]
"Top List:"	["0000001214", "GRO85051"]
"Top List:"	["0000001219", "DAI22896"]
"Top List:"	["0000001220", "GRO81087"]
"Top List:"	["0000001261", "DAI31081"]
"Top List:"	["0000001275", "GRO15017"]
"Top List:"	["0000001289", "ELE91337"]
"Top List:"	["0000001290", "DAI43223"]
"Top List:"	["0000001295", "SNA96271"]
"Top List:"	["0000001311", "ELE59935"]
"Top List:"	["0000001316", "DAI88807"]
"Top List:"	["0000001316", "ELE74482"]
"Top List:"	["0000001321", "GRO61133"]
"Top List:"	["0000001345", "ELE56788"]
"Top List:"	["0000001352", "GRO38814"]
"Top List:"	["0000001390", "SNA90094"

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140733.870000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
Counters: 3
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
		Reducer_Call_Num=12592
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num=12592
Counters: 2
	group
		Mapper2_Call_Num=12592
		Reducer2_Call_Num=1
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140733.870000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_top_50.z001c9v.20160729.140733.870000...


In [21]:
!python km_2_3_shoppingcart_uniques.py --jobconf mapred.reduce.tasks=2 ProductPurchaseData.txt

"Total Uniques:"	12592


No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140739.077000
Running step 1 of 2...
Counters: 2
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
Counters: 3
	group
		Combiner_Call_Num=17745
		Mapper_Call_Num=31101
		Reducer_Call_Num=12592
Running step 2 of 2...
Counters: 1
	group
		Mapper2_Call_Num:=25184
Counters: 2
	group
		Mapper2_Call_Num:=25184
		Reducer2_Call_Num:=12593
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140739.077000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_uniques.z001c9v.20160729.140739.077000...


In [22]:
!python km_2_3_shoppingcart_largest.py --jobconf mapred.reduce.tasks=2 ProductPurchaseData.txt

"Largest Basket:"	[["DAI11223", "ELE54102", "SNA56249", "SNA30755", "FRO80039", "SNA53220"], 6]
"Largest Basket:"	[["DAI11238", "SNA82274", "SNA96466", "GRO88324", "SNA43409", "FRO35729", "GRO83463", "GRO30912", "ELE34234", "ELE26753", "ELE45560", "ELE99887", "ELE23393", "SNA31446", "SNA40784", "GRO71621"], 16]
"Largest Basket:"	[["DAI11927", "FRO60829", "FRO75586", "ELE49863", "SNA91739", "GRO76157", "FRO73019", "SNA66583", "ELE30182", "DAI22240", "GRO91660", "DAI91705", "GRO81087", "SNA92303", "DAI50189", "GRO48219", "GRO61273"], 17]
"Largest Basket:"	[["DAI12680", "FRO91015", "ELE53126", "SNA56249", "ELE37770", "DAI70456", "SNA63157", "FRO14026", "DAI55911", "SNA37803", "GRO19500", "GRO46854", "FRO40251", "FRO77785", "ELE62598", "DAI75981", "SNA80324", "FRO75586"], 18]
"Largest Basket:"	[["DAI13776", "SNA90094", "GRO44993", "DAI43868", "SNA82528", "FRO73056", "GRO30386", "FRO98184", "SNA96271", "GRO16201", "SNA87654", "SNA61957", "GRO50132", "GRO64900", "FRO62184", "DAI62779", "DAI8

No configs found; falling back on auto-configuration
Creating temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140744.732000
Running step 1 of 1...
Counters: 1
	group
		Mapper_Call_Num=31101
Counters: 2
	group
		Mapper_Call_Num=31101
		Reducer_Call_Num=30697
Streaming final output from c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140744.732000\output...
Removing temp directory c:\users\z001c9v\appdata\local\temp\km_2_3_shoppingcart_largest.z001c9v.20160729.140744.732000...


In [ ]:
# Results: No difference in the run process for part #1, part #2, part#3 using 2 reducers.

# HW2.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs


In [23]:
# Calculating the total number of pairs for the ProductPurchaseData.txt data set.
import math

def item_combo(n):
    pairs = n**2 / 2
    return pairs

num_pairs = item_combo(12592)
print "This is the total number of item pairs:" + str(num_pairs)    
    

This is the total number of item pairs:79279232


### Task: pg.195 Minin of Massive Data Sets:
> "For each basket we use a double loop to generate all the pairs.  Each time we generate a pair, we add 1 to its count. At the end, we examine all pairs and see which have counts that exceed the suppor theshold s; these are the frequent pairs.

**Support Threshold = 100**

In [24]:

add = open("PPD_shortlist.txt" , 'w')
base = open("ProductPurchaseData.txt" , 'r')
counter = 0 
  
for line in base:
    counter += 1
    if counter < 100 :
        add.write(line)

add.close()
base.close()

In [25]:
%%writefile km_hw_2_4_Apriori.py 
from mrjob.job import MRJob
from mrjob.step import MRStep
import string , re

counter = 0

class Apriori_itemsets(MRJob):
    
    '''
    def steps(self):
        return [
            MRStep ( mapper = self.mapper , 
                     combiner = self.combiner , 
                     reducer = self.reducer ) ,
            MRStep ( mapper = self.mapper2 
                   )
        ]
        
    '''#,
    #                reducer = self.reducer2 

    def __init__(self, *args, **kargs) :
        super(Apriori_itemsets,self).__init__(*args, **kargs)
        self.L1 = {}
        self.L2 = {}
        self.item_list = {}
        self.bags = []


    def mapper(self , _ , line) :
        self.increment_counter('group' , 'Mapper_Call_Num' , 1)
        
        bags = line.strip().split(' ') 
        #basketsize = len(bags)
        #print bags , len(bags)
        #for item in bags:
         #   #print item , 1
        #  yield item , 1
        for i in bags:
            # i not in self.item_list.keys() : return
            #counter = 0 
            for j in bags:
                #counter += 1
                #if counter > 5 : break
                #if j not in self.item_list.keys() : return
                while i <> j:
                    #self.L2[i+j] += 1
                    yield i+j , 1         
            
    def combiner(self, key, value):
        self.increment_counter('group' , 'Combiner_Call_Num' , 1)
        yield (key , sum(value))
    
    def reducer(self, key, value):
        #print key , sum(value)
        self.increment_counter('group' , 'Reducer_Call_Num' , 1)
        total = sum(value)
        #print key , total
        if total >= 10 : 
            #print key , total
            self.item_list[key] = total
        yield 'None' , self.item_list

            
    '''
    def mapper2(self, key , line):
        self.increment_counter('group' , 'Mapper2_Call_Num' ,1)
        print  key , line
        print '\n'
        #bags = line.strip().split(' ')
        #basketsize = len(bags)
        #print self.item_list

        for i in bags:
            if i not in self.item_list.keys() : return
            counter = 0 
            for j in bags:
                if counter > 5 : break
                if j not in self.item_list.keys() : return
                while i <> j:
                    self.L2[i+j] +1
                    print i+j , 1 
            #print item , 1
            #yield item , 1

        # yield None , (key , value)
        
    
    def reducer2(self, key , value):
        self.increment_counter('group' , 'Reducer2_Call_Num' , 1)      
        #for word, num in value:
        #    yield num , word
        #print 'reducer2:' , (key , list(value) )
        final_list = []
        for p in value :
            final_list.append(p)
            final_list_bot = final_list[0:9]
            final_list_top = final_list[-51:]
        
        for num , word in final_list_bot : 
            yield 'Bottom List:' , (num , word)
        for num , word in final_list_top :
            yield 'Top List:' , (num , word)
            
        #yield final_list[-51:]
        #yield '\n'
        #yield final_list[0:9]
            
        #print final_list[-51:]
        #print '\n'
        #print final_list[0:9]
    '''
               
        
if __name__ == "__main__" :
    Apriori_itemsets.run()


Overwriting km_hw_2_4_Apriori.py


In [ ]:
!python km_hw_2_4_Apriori.py ProductPurchaseData.txt 